### Notebook for generating the MnAu2 cell structure inputs for the QE calculations

Following the steps in the bcc-fe-exchange-model notebook, read in an initial .cif file (6-atoms) and 
combine with magnetic patterns. Translates in the x,y direction to increase the number of Mn atoms to allow 
for more magnetic patterns. Configuration files saved to working folder.

- 6 atom configuration allows for NM, FM and AFM-A. 

In [2]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D

from ruamel.yaml import YAML
yaml = YAML()

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

In [11]:
cif_filepath = "mnau2-6.cif"
structure_mnau2 = from_file(structure_file=cif_filepath)
print(structure_mnau2)

Full Formula (Mn2 Au4)
Reduced Formula: MnAu2
abc   :   3.394839   3.394839   8.880636
angles:  90.000000  90.000000  90.000000
Sites (6)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Mn    0    0    0
  1  Au    0.5  0.5  0.840319
  2  Au    0.5  0.5  0.159681
  3  Mn    0.5  0.5  0.5
  4  Au    0    0    0.340319
  5  Au    0    0    0.659681


In [5]:
pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}

magnetic_patterns = yaml.load(Path("magnetic_patterns-6.yml"))
pd.DataFrame(magnetic_patterns["6atoms"]).style \
    .set_table_styles(html_table_style) \
    .hide_index()

nm,f-type,a-type
0,1,-1
0,0,0
0,0,0
0,1,1
0,0,0
0,0,0


In [6]:
y={}
y['nm'] = magnetic_patterns['6atoms']['nm']
y['f-type'] = magnetic_patterns['6atoms']['f-type']
y['a-type'] = magnetic_patterns['6atoms']['a-type']
new6 = structure_mnau2.copy(y)
new6.make_supercell([5,5,5])
xyz = new6.cart_coords
supercell = pd.concat([pd.DataFrame(new6.species, columns=["species"]),
                          pd.DataFrame(new6.cart_coords, columns=["x","y","z"]), 
                          pd.DataFrame(new6.site_properties["nm"], columns=["nm"]) , 
                          pd.DataFrame(new6.site_properties["f-type"], columns=["f-type"]) ,
                          pd.DataFrame(new6.site_properties["a-type"], columns=["a-type"]) ,
                      ],
                          axis=1, )

supercell.to_csv('mnau2-6.xyz', header=True, index=False, sep='\t')

In [8]:
rotate_cell_45_degrees = SupercellTransformation(
    scaling_matrix=[[1, 1, 0],
                    [1, -1, 0],
                    [0, 0, 1]],
)

rotated_structure_mnau2 = rotate_cell_45_degrees.apply_transformation(structure_mnau2)
print(rotated_structure_mnau2)

Full Formula (Mn4 Au8)
Reduced Formula: MnAu2
abc   :   4.801027   4.801027   8.880636
angles:  90.000000  90.000000  90.000000
Sites (12)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Mn    0    0    0
  1  Mn    0.5  0.5  0
  2  Au    0.5  0    0.840319
  3  Au    1    0.5  0.840319
  4  Au    0.5  0    0.159681
  5  Au    1    0.5  0.159681
  6  Mn    0.5  0    0.5
  7  Mn    1    0.5  0.5
  8  Au    0    0    0.340319
  9  Au    0.5  0.5  0.340319
 10  Au    0    0    0.659681
 11  Au    0.5  0.5  0.659681


In [9]:
rotated_structure_mnau2.to(filename="mnau2-12.cif")

In [10]:
pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}

magnetic_patterns = yaml.load(Path("magnetic_patterns.yml"))
pd.DataFrame(magnetic_patterns["12atoms"]).style \
    .set_table_styles(html_table_style) \
    .hide_index()

nm,f-type,a-type,g-type
0,1,1,1
0,0,0,0
0,0,0,0
0,1,1,-1
0,0,0,0
0,0,0,0
0,1,-1,1
0,0,0,0
0,0,0,0
0,1,-1,-1


In [44]:
x={}
x['nm'] = magnetic_patterns['12atoms']['nm']
x['f-type'] = magnetic_patterns['12atoms']['f-type']
x['a-type'] = magnetic_patterns['12atoms']['a-type']
x['g-type'] = magnetic_patterns['12atoms']['g-type']
new = structure_mnau2.copy(x)
new.make_supercell([5,5,5])
xyz = new.cart_coords
supercell = pd.concat([pd.DataFrame(new.species, columns=["species"]),
                          pd.DataFrame(new.cart_coords, columns=["x","y","z"]), 
                          pd.DataFrame(new.site_properties["nm"], columns=["nm"]) , 
                          pd.DataFrame(new.site_properties["f-type"], columns=["f-type"]) ,
                          pd.DataFrame(new.site_properties["a-type"], columns=["a-type"]) ,
                          pd.DataFrame(new.site_properties["g-type"], columns=["g-type"]) 
                      ],
                          axis=1, )

supercell.to_csv('mnau2.xyz', header=True, index=False, sep='\t')